In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
!pip install -q -U transformers accelerate peft trl bitsandbytes wandb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import json
import torch
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import wandb

2025-10-27 11:11:08.514060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761563468.746536      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761563468.810784      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

In [5]:
def categorize_examples(data):
    """Categorize examples by topic for stratified splitting"""
    
    categories = {
        'ensakh_programs': [],
        'program_comparisons': [],
        'ensakh_info': [],
        'ml_concepts': [],
        'cybersecurity': [],
        'software_eng': [],
        'data_eng': [],
        'electrical_eng': [],
        'general_tech': [],
        'other': []
    }
    
    # Keywords for categorization
    program_keywords = ['GI', 'IID', 'GE', 'IRIC', 'GPEE', 'MGSI', 'GRT', 'Computer Engineering', 
                        'Génie Informatique', 'what is', 'c\'est quoi']
    comparison_keywords = ['difference', 'compare', 'vs', 'or', 'which is better', 'différence']
    ensakh_keywords = ['ensakh', 'admission', 'contact', 'email', 'programs', 'student life', 
                       'research', 'facilities']
    ml_keywords = ['learning', 'neural', 'cnn', 'rnn', 'transformer', 'overfitting', 
                   'backpropagation', 'gradient']
    cyber_keywords = ['encryption', 'firewall', 'phishing', 'vpn', 'ddos', 'xss', 
                      'penetration testing', 'sql injection']
    software_keywords = ['programming', 'api', 'docker', 'git', 'microservices', 
                         'design patterns', 'ci/cd']
    data_keywords = ['data', 'etl', 'pipeline', 'warehouse', 'feature engineering', 
                     'data lake']
    electrical_keywords = ['motor', 'transformer', 'power', 'circuit', 'plc', 'scada', 
                           'electrical', 'voltage']
    tech_keywords = ['iot', 'blockchain', 'quantum', '5g', 'cloud computing', 'robotics']
    
    for idx, example in enumerate(data):
        prompt_lower = example['prompt'].lower()
        
        # Categorize
        if any(kw.lower() in prompt_lower for kw in comparison_keywords):
            categories['program_comparisons'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in ensakh_keywords):
            categories['ensakh_info'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in program_keywords) and \
             not any(kw.lower() in prompt_lower for kw in comparison_keywords):
            categories['ensakh_programs'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in ml_keywords):
            categories['ml_concepts'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in cyber_keywords):
            categories['cybersecurity'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in software_keywords):
            categories['software_eng'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in data_keywords):
            categories['data_eng'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in electrical_keywords):
            categories['electrical_eng'].append(idx)
        elif any(kw.lower() in prompt_lower for kw in tech_keywords):
            categories['general_tech'].append(idx)
        else:
            categories['other'].append(idx)
    
    return categories


def remove_similar_examples(train_indices, test_indices, data, threshold=0.85):
    """Remove test examples that are too similar to training examples"""
    
    train_texts = [data[i]['prompt'] for i in train_indices]
    test_texts = [data[i]['prompt'] for i in test_indices]
    
    # Compute TF-IDF similarity
    vectorizer = TfidfVectorizer()
    train_vectors = vectorizer.fit_transform(train_texts)
    test_vectors = vectorizer.transform(test_texts)
    
    similarities = cosine_similarity(test_vectors, train_vectors)
    
    # Keep only test examples that are not too similar to training
    filtered_test_indices = []
    for i, test_idx in enumerate(test_indices):
        max_sim = similarities[i].max()
        if max_sim < threshold:
            filtered_test_indices.append(test_idx)
    
    return filtered_test_indices


def stratified_split(data, test_size=0.15, val_size=0.05):
    """
    Create stratified train/val/test split
    - test_size: 15% for true unseen evaluation
    - val_size: 5% for overfitting monitoring during training
    - train_size: 80% for learning
    """
    
    categories = categorize_examples(data)
    
    train_indices = []
    val_indices = []
    test_indices = []
    
    np.random.seed(42)
    
    for category, indices in categories.items():
        if len(indices) == 0:
            continue
            
        # Shuffle indices
        shuffled = np.random.permutation(indices).tolist()
        
        # Calculate splits
        n_test = max(1, int(len(shuffled) * test_size))
        n_val = max(1, int(len(shuffled) * val_size))
        
        # Split
        test_indices.extend(shuffled[:n_test])
        val_indices.extend(shuffled[n_test:n_test + n_val])
        train_indices.extend(shuffled[n_test + n_val:])
        
        print(f"📂 {category:20s}: Train={len(shuffled[n_test + n_val:]):3d} | "
              f"Val={len(shuffled[n_test:n_test + n_val]):2d} | "
              f"Test={len(shuffled[:n_test]):2d}")
    
    # Remove test examples too similar to training
    print("\n🔍 Removing similar examples from test set...")
    original_test_size = len(test_indices)
    test_indices = remove_similar_examples(train_indices, test_indices, data)
    print(f"   Filtered: {original_test_size} → {len(test_indices)} test examples")
    
    return train_indices, val_indices, test_indices

In [6]:
def load_and_split_data(jsonl_path="/kaggle/input/finaldata/input.jsonl"):
    """Load data and create intelligent splits"""
    
    # Load data with robust error handling for JSONL
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Try multiple parsing strategies
    # Strategy 1: Standard JSONL (one JSON object per line)
    try:
        for line_num, line in enumerate(content.strip().split('\n'), 1):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"⚠️  Warning: Malformed JSON on line {line_num}: {str(e)[:50]}")
                continue
    except Exception as e:
        print(f"⚠️  Standard JSONL parsing failed: {e}")
        data = []
    
    # Strategy 2: If Strategy 1 fails or gets no data, try parsing as JSON array
    if len(data) == 0:
        print("🔄 Trying alternative parsing: JSON array format...")
        try:
            data = json.loads(content)
            if not isinstance(data, list):
                data = [data]
        except json.JSONDecodeError:
            print("❌ Could not parse file as JSON array either")
            
    # Strategy 3: Try splitting by "}{"  (concatenated JSON objects)
    if len(data) == 0:
        print("🔄 Trying alternative parsing: Concatenated JSON objects...")
        try:
            # Add commas between objects and wrap in array
            fixed_content = content.replace('}{', '},{')
            if not fixed_content.startswith('['):
                fixed_content = '[' + fixed_content + ']'
            data = json.loads(fixed_content)
        except json.JSONDecodeError:
            print("❌ Could not parse concatenated JSON objects")
    
    if len(data) == 0:
        raise ValueError("❌ Failed to load any data from JSONL file. Please check file format.")
    
    print(f"✅ Loaded {len(data)} examples\n")
    
    # Create stratified split
    train_idx, val_idx, test_idx = stratified_split(data)
    
    # ✅ Format conversations (sequences will be truncated by tokenizer automatically)
    def format_conversation(example):
        return {
            "text": f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are ENSAKH Assistant, a helpful academic chatbot for ENSAKH (École Nationale des Sciences Appliquées de Khouribga). 
Answer questions about ENSAKH programs, admissions, facilities, and technical topics accurately and helpfully.
Always respond in the same language as the question (English, French, or Darija).<|eot_id|><|start_header_id|>user<|end_header_id|>

{example['prompt']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{example['response']}<|eot_id|>"""
        }
    
    # Create datasets
    train_data = [format_conversation(data[i]) for i in train_idx]
    val_data = [format_conversation(data[i]) for i in val_idx]
    test_data = [format_conversation(data[i]) for i in test_idx]
    
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    test_dataset = Dataset.from_list(test_data)
    
    # Save test set separately for later evaluation
    test_raw = [{'prompt': data[i]['prompt'], 'response': data[i]['response']} 
                for i in test_idx]
    with open('/kaggle/working/test_set.jsonl', 'w', encoding='utf-8') as f:
        for item in test_raw:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"\n📊 Final Split:")
    print(f"   Train: {len(train_dataset)} examples (80%)")
    print(f"   Val:   {len(val_dataset)} examples (5%)")
    print(f"   Test:  {len(test_dataset)} examples (15%)")
    print(f"\n💾 Test set saved to '/kaggle/working/test_set.jsonl'\n")
    
    return train_dataset, val_dataset, test_dataset

In [7]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

# Get the token from Kaggle Secrets
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Login to HuggingFace
login(token=hf_token)

print("✅ Successfully logged in to HuggingFace!")
print("🔓 You now have access to gated models like LLaMA 3.1")


✅ Successfully logged in to HuggingFace!
🔓 You now have access to gated models like LLaMA 3.1


In [8]:
class Config:
    # Model Configuration
    MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    
    # LoRA Configuration - OPTIMIZED for 2K examples
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.1
    LORA_TARGET_MODULES = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
    
    # Training Arguments - ANTI-OVERFITTING OPTIMIZED
    OUTPUT_DIR = "/kaggle/working/ensakh-assistant-lora"
    NUM_EPOCHS = 3
    BATCH_SIZE = 4
    GRADIENT_ACCUMULATION = 2
    LEARNING_RATE = 2e-4
    WARMUP_RATIO = 0.1
    LR_SCHEDULER = "cosine"
    WEIGHT_DECAY = 0.01
    MAX_SEQ_LENGTH = 512  # ✅ Keep this for tokenizer configuration
    
    # Evaluation & Early Stopping
    EVAL_STRATEGY = "steps"
    EVAL_STEPS = 50
    SAVE_STEPS = 50
    SAVE_TOTAL_LIMIT = 2
    LOAD_BEST_AT_END = True
    METRIC_FOR_BEST = "eval_loss"
    
    # Logging
    LOGGING_STEPS = 10
    REPORT_TO = "none"  
    
    # 4-bit Quantization
    USE_4BIT = True
    BNB_4BIT_COMPUTE_DTYPE = "float16"
    BNB_4BIT_QUANT_TYPE = "nf4"
    USE_NESTED_QUANT = True

In [9]:
def setup_model_and_tokenizer():
    """Setup LLaMA 3.1 with 4-bit quantization and LoRA"""
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=Config.USE_4BIT,
        bnb_4bit_quant_type=Config.BNB_4BIT_QUANT_TYPE,
        bnb_4bit_compute_dtype=getattr(torch, Config.BNB_4BIT_COMPUTE_DTYPE),
        bnb_4bit_use_double_quant=Config.USE_NESTED_QUANT,
    )
    
    print("🔄 Loading LLaMA 3.1:8B...")
    model = AutoModelForCausalLM.from_pretrained(
        Config.MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    
    model = prepare_model_for_kbit_training(model)
    
    peft_config = LoraConfig(
        r=Config.LORA_R,
        lora_alpha=Config.LORA_ALPHA,
        lora_dropout=Config.LORA_DROPOUT,
        target_modules=Config.LORA_TARGET_MODULES,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    model = get_peft_model(model, peft_config)
    
    tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"✨ Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)\n")
    
    return model, tokenizer

In [10]:
def get_training_args():
    """Training arguments optimized to prevent overfitting"""
    
    from transformers import TrainingArguments
    
    return TrainingArguments(
        output_dir=Config.OUTPUT_DIR,
        num_train_epochs=Config.NUM_EPOCHS,
        per_device_train_batch_size=Config.BATCH_SIZE,
        per_device_eval_batch_size=Config.BATCH_SIZE,
        gradient_accumulation_steps=Config.GRADIENT_ACCUMULATION,
        learning_rate=Config.LEARNING_RATE,
        weight_decay=Config.WEIGHT_DECAY,
        warmup_ratio=Config.WARMUP_RATIO,
        lr_scheduler_type=Config.LR_SCHEDULER,
        
        eval_strategy=Config.EVAL_STRATEGY,
        eval_steps=Config.EVAL_STEPS,
        save_strategy="steps",
        save_steps=Config.SAVE_STEPS,
        save_total_limit=Config.SAVE_TOTAL_LIMIT,
        load_best_model_at_end=Config.LOAD_BEST_AT_END,
        metric_for_best_model=Config.METRIC_FOR_BEST,
        greater_is_better=False,
        
        logging_steps=Config.LOGGING_STEPS,
        report_to=Config.REPORT_TO,
        
        fp16=True,
        optim="paged_adamw_8bit",
        gradient_checkpointing=True,
        
        seed=42,
    )

In [11]:
def train():
    """Main training function with proper data splits"""
    
    # Load and split data
    train_dataset, val_dataset, test_dataset = load_and_split_data()
    
    # Setup model & tokenizer
    model, tokenizer = setup_model_and_tokenizer()
    
    # Training arguments
    training_args = get_training_args()
    
    # SFT Trainer - FIXED VERSION (compatible with latest trl)
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        processing_class=tokenizer,  
        args=training_args,
    )
    
    # Train!
    print("🚀 Starting training...\n")
    trainer.train()
    
    # Save final model
    print("\n💾 Saving model...")
    trainer.save_model(Config.OUTPUT_DIR)
    tokenizer.save_pretrained(Config.OUTPUT_DIR)
    
    print("✅ Training complete!\n")
    
    # Evaluate on validation set
    val_results = trainer.evaluate()
    print(f"📊 Validation Loss: {val_results['eval_loss']:.4f}")
    
    return trainer, test_dataset

In [12]:
def evaluate_on_test_set(model_path=Config.OUTPUT_DIR, test_jsonl='/kaggle/working/test_set.jsonl'):
    """Evaluate the fine-tuned model on the held-out test set"""
    
    print("="*60)
    print("🧪 EVALUATING ON UNSEEN TEST SET")
    print("="*60)
    
    # Load model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.float16
    )
    
    # Load test set
    with open(test_jsonl, 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f]
    
    print(f"\n📋 Testing on {len(test_data)} unseen examples\n")
    
    # Sample evaluation
    sample_indices = np.random.choice(len(test_data), min(10, len(test_data)), replace=False)
    
    for idx in sample_indices:
        example = test_data[idx]
        prompt = example['prompt']
        expected = example['response']
        
        # Generate response
        conversation = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are ENSAKH Assistant, a helpful academic chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
        inputs = tokenizer(conversation, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated = generated.split("assistant<|end_header_id|>")[-1].strip()
        
        print(f"\n{'='*60}")
        print(f"❓ Question: {prompt}")
        print(f"\n🤖 Generated:\n{generated[:200]}...")
        print(f"\n✅ Expected:\n{expected[:200]}...")
    
    print(f"\n{'='*60}")
    print("✅ Test evaluation complete!")

In [13]:
def generate_response(prompt, model, tokenizer, max_new_tokens=256):
    """Generate response from fine-tuned model"""
    
    conversation = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are ENSAKH Assistant, a helpful academic chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    inputs = tokenizer(conversation, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("assistant<|end_header_id|>")[-1].strip()
    
    return response

In [14]:
if __name__ == "__main__":
    # Train the model
    trainer, test_dataset = train()
    
    # Evaluate on test set
    evaluate_on_test_set()
    
    print("\n" + "="*60)
    print("🎉 ALL DONE! Model trained and evaluated!")
    print("="*60)
    print("\n📁 Files created:")
    print("   - ensakh-assistant-lora/ (trained model)")
    print("   - test_set.jsonl (held-out test examples)")
    print("\n💡 Next steps:")
    print("   1. Review test set performance")
    print("   2. Build RAG system for document retrieval")
    print("   3. Deploy with FastAPI")

⚠️  Warning: Malformed JSON on line 10: Extra data: line 1 column 645 (char 644)
⚠️  Warning: Malformed JSON on line 1801: Invalid \escape: line 1 column 1735 (char 1734)
✅ Loaded 1995 examples

📂 ensakh_programs     : Train=302 | Val=18 | Test=56
📂 program_comparisons : Train=454 | Val=28 | Test=85
📂 ensakh_info         : Train=256 | Val=15 | Test=47
📂 ml_concepts         : Train=  8 | Val= 1 | Test= 1
📂 cybersecurity       : Train=  0 | Val= 0 | Test= 1
📂 software_eng        : Train= 44 | Val= 2 | Test= 7
📂 data_eng            : Train= 24 | Val= 1 | Test= 4
📂 electrical_eng      : Train=  5 | Val= 1 | Test= 1
📂 general_tech        : Train=  6 | Val= 1 | Test= 1
📂 other               : Train=502 | Val=31 | Test=93

🔍 Removing similar examples from test set...
   Filtered: 296 → 206 test examples

📊 Final Split:
   Train: 1601 examples (80%)
   Val:   98 examples (5%)
   Test:  206 examples (15%)

💾 Test set saved to '/kaggle/working/test_set.jsonl'

🔄 Loading LLaMA 3.1:8B...


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...


💾 Saving model...
✅ Training complete!



📊 Validation Loss: 0.4059
🧪 EVALUATING ON UNSEEN TEST SET


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


📋 Testing on 206 unseen examples


❓ Question: What is Advanced Industrial Control?

🤖 Generated:
system

You are ENSAKH Assistant, a helpful academic chatbot.user

What is Advanced Industrial Control?assistant

**Advanced Industrial Control** is S5 module that builds on industrial automation conc...

✅ Expected:
It is a module in S3 that deepens control and automatic regulation techniques for industrial processes....

❓ Question: what is the registration when open ?? please

🤖 Generated:
system

You are ENSAKH Assistant, a helpful academic chatbot.user

what is the registration when open?? pleaseassistant

For specific registration dates, please contact ENSAKH at contact.ensak@usms.ma...

✅ Expected:
I'm not fully sure what you mean by “registration”. Do you mean program admission, course enrollment, or account activation? If you clarify, I can guide you. For admissions, check the official dates o...

❓ Question: Who represents students?

🤖 Generated:
system

You are ENSAKH Assistant

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

def merge_lora_to_base_model(
    base_model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    lora_adapter_path="/kaggle/working/ensakh-assistant-lora",
    output_path="/kaggle/working/ensakh-llama-merged",
    push_to_hub=False,
    hub_model_name="your-username/ensakh-llama-3.1-8b"
):
    """
    Merge LoRA adapters with base model to create a standalone model.
    
    This creates a FULL model that contains:
    - All original LLaMA 3.1 knowledge
    - Your fine-tuned ENSAKH knowledge
    
    Args:
        base_model_name: Original LLaMA model
        lora_adapter_path: Path to your trained LoRA adapters
        output_path: Where to save the merged model
        push_to_hub: Whether to upload to HuggingFace Hub
        hub_model_name: Name for the model on HuggingFace Hub
    """
    
    print("="*60)
    print("🔗 MERGING LORA ADAPTERS WITH BASE MODEL")
    print("="*60)
    
    # Step 1: Load base model
    print("\n📥 Loading base model (LLaMA 3.1:8B)...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # Step 2: Load LoRA adapters
    print("📥 Loading LoRA adapters (ENSAKH knowledge)...")
    model_with_lora = PeftModel.from_pretrained(
        base_model,
        lora_adapter_path,
        torch_dtype=torch.float16
    )
    
    # Step 3: Merge LoRA weights into base model
    print("🔄 Merging LoRA weights into base model...")
    print("   (This combines both knowledge bases)")
    merged_model = model_with_lora.merge_and_unload()
    
    # Step 4: Load tokenizer
    print("📥 Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(lora_adapter_path)
    
    # Step 5: Save merged model locally
    print(f"\n💾 Saving merged model to {output_path}...")
    merged_model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    
    print("✅ Merged model saved successfully!")
    
    # Step 6: (Optional) Push to HuggingFace Hub
    if push_to_hub:
        print(f"\n📤 Pushing to HuggingFace Hub as '{hub_model_name}'...")
        merged_model.push_to_hub(hub_model_name)
        tokenizer.push_to_hub(hub_model_name)
        print("✅ Model uploaded to HuggingFace Hub!")
    
    print("\n" + "="*60)
    print("🎉 MERGE COMPLETE!")
    print("="*60)
    print(f"\n📦 Your unified model is ready:")
    print(f"   Location: {output_path}")
    print(f"   Contains: LLaMA 3.1 knowledge + ENSAKH knowledge")
    print(f"   Size: ~8B parameters (same as original)")
    print(f"\n💡 You can now use this model without LoRA adapters!")
    
    return merged_model, tokenizer


def test_merged_model(model, tokenizer):
    """Test the merged model with sample questions"""
    
    print("\n" + "="*60)
    print("🧪 TESTING MERGED MODEL")
    print("="*60)
    
    test_questions = [
        "What is GI at ENSAKH?",
        "Explain what is machine learning",
        "What are the admission requirements for ENSAKH?",
        "Compare Python and Java"
    ]
    
    for question in test_questions:
        print(f"\n❓ Question: {question}")
        
        conversation = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are ENSAKH Assistant, a helpful academic chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
        
        inputs = tokenizer(conversation, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.split("assistant<|end_header_id|>")[-1].strip()
        
        print(f"🤖 Answer: {response[:300]}...")
        print("-"*60)


def load_merged_model_for_inference(model_path="/kaggle/working/ensakh-llama-merged"):
    """
    Load the merged model for inference (no LoRA needed!)
    
    This is how you'll use the model in production.
    """
    
    print("📥 Loading merged model for inference...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    print("✅ Model loaded! Ready for inference.")
    return model, tokenizer


# =============================================================================
# USAGE EXAMPLE
# =============================================================================

if __name__ == "__main__":
    # After training is complete, run this:
    
    # 1. Merge LoRA with base model
    merged_model, tokenizer = merge_lora_to_base_model(
        base_model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
        lora_adapter_path="/kaggle/working/ensakh-assistant-lora",
        output_path="/kaggle/working/ensakh-llama-merged",
        push_to_hub=False  # Set to True if you want to upload to HF Hub
    )
    
    # 2. Test the merged model
    test_merged_model(merged_model, tokenizer)
    
    print("\n✅ All done! Your model is ready for deployment.")
    print("\n📝 Note: The merged model:")
    print("   • Contains ALL original LLaMA 3.1 capabilities")
    print("   • Plus your specialized ENSAKH knowledge")
    print("   • Can be used without PEFT/LoRA libraries")
    print("   • Perfect for deployment!")

🔗 MERGING LORA ADAPTERS WITH BASE MODEL

📥 Loading base model (LLaMA 3.1:8B)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

📥 Loading LoRA adapters (ENSAKH knowledge)...
🔄 Merging LoRA weights into base model...
   (This combines both knowledge bases)
📥 Loading tokenizer...

💾 Saving merged model to /kaggle/working/ensakh-llama-merged...


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3970: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Merged model saved successfully!

🎉 MERGE COMPLETE!

📦 Your unified model is ready:
   Location: /kaggle/working/ensakh-llama-merged
   Contains: LLaMA 3.1 knowledge + ENSAKH knowledge
   Size: ~8B parameters (same as original)

💡 You can now use this model without LoRA adapters!

🧪 TESTING MERGED MODEL

❓ Question: What is GI at ENSAKH?
🤖 Answer: system

You are ENSAKH Assistant, a helpful academic chatbot.user

What is GI at ENSAKH?assistant

Génie Informatique (GI) is one of the engineering programs offered by the Mathématiques et Informatique department. It focuses on training engineers with expertise in software engineering, programming,...
------------------------------------------------------------

❓ Question: Explain what is machine learning
🤖 Answer: system

You are ENSAKH Assistant, a helpful academic chatbot.user

Explain what is machine learningassistant

**Machine Learning** is a subset of AI that enables machines to learn from data.

**Key Idea:**
"Give me data, I'll f